<a href="https://colab.research.google.com/github/heitorcfelix/Agente-Revisor-de-Codigo/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai google-adk PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
from github import Github

# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(userdata.get('GITHUB_API_KEY'))

# First create a Github instance:
g = Github(auth=auth)

# Then play with your Github privete objects:
for repo in g.get_user().get_repos():
    print(repo.name)

# To close connections after use
g.close()

InsPLAD
PLAD
skip-ganomaly
if702-redes-neurais
leap-talk
DOLL
ObjLocalisation
2018.2-codigo
ABC_Tracking
AR.js
CalculosFarmaceuticos
Cashless-01
ComputacaoBioinspirada
Cracking-the-Code
Dataset-Imbalance-Graphical-Support-Tool
DeOldify
exercicio-calculadora
exercicio-rss
GrupoMF
heitorcfelix.github.io
iaLAB-HUB
incognia
ivy
logllm
logllm-front
Monitor-de-Consumo-Energetico
NUDFACFerramentas
OGNet
OpenWeatherService
P3-4
P3-6
Path_Tracer
PetLab-Lika
public-insulator-datasets
python-terraform
Pytorch-UNet
se-eu-nao-posso-ninguem-pode
singleshot6Dpose
TableOn
tg
VISRecorder
if678
robocode-Justice_League


In [7]:
# prompt: access AR.js repository
repo = g.get_user().get_repo("OpenWeatherService")
repo.name

# print readme with ipython md
from IPython.display import display, Markdown
readme = repo.get_contents("README.md")
display(to_markdown(readme.decoded_content.decode("utf-8")))


> # OpenWeatherService
> ### Service that collects data from an Open Weather API and stores it as JSON data in a PostgreSQL DB
> 
> ### Requirements
> - Docker >= 24.0.5
> 
> ### How to Build
> 
> - Create an [OpenWeather Account](https://openweathermap.org/) and Generate a API Key
> - Put the key and all sensitive values in secrets folder
>     - [AppID](./secrets/appid.env)
>     - [DB URI](./secrets/database_url.env)
>     - [DB USER](./secrets/database_user.env)
>     - [DB PASSWORD](./secrets/database_password.env)
> 
> - Run
> ```sh
> docker compose up -d
> ```
> 
> ### How to Run
> Receives a user defined ID, collect weather data from OpenWeather API and store:
> - The user defined ID (needs to be unique for each request)
> - Datetime of request
> - JSON data with:
>     - City ID
>     - Temperature in Celsius
>     - Humidity
> ```sh
> curl -XPOST -H 'Content-Type: application/x-www-form-urlencoded' -d 'user_id=1' 'http://localhost:8000'
> ```
> 
> Receives the user defined ID, returns with the percentage of the POST progress ID (collected cities completed) until the current moment.
> ```sh
> curl -XGET 'http://localhost:8000?user_id=1'
> ```
> 
> ### How to Test
> ```sh
> python test/test_app.py
> ```
> The test will try all 161 Appendix A cities, it should take considerable time. To change the list of cities, change in [city_codes](./app/main/city_codes.py).

In [8]:
def _listar_todos_os_arquivos(repo, path="/"):
    arquivos = []
    conteudo = repo.get_contents(path)
    while conteudo:
        arquivo = conteudo.pop(0)
        if arquivo.type == "dir":
            conteudo.extend(repo.get_contents(arquivo.path))
        else:
            arquivos.append(arquivo)
    return arquivos


In [23]:
##########################################
# --- Agente 1: Agente interpletador de arquivo de código --- #
##########################################
def agente_leitor(arquivo):

    analista_codigo = Agent(
        name="analista_codigo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de análise de código. Sua tarefa é analisar **um arquivo de código por vez**, recebido como entrada,
        e gerar uma explicação clara e objetiva sobre o que esse arquivo faz. A explicação não deve conter o conteúdo no arquivo, apenas o objetibo geral do arquivo.

        Para cada arquivo que receber:
        - Indique o nome do arquivo.
        - Explique qual é o propósito geral do arquivo dentro de um projeto de software.
        - Destaque e explique as principais funções, classes, componentes ou blocos de lógica presentes.
        - Se possível, comente como esse arquivo pode se relacionar com outras partes de um sistema típico (ex: "essa função pode ser usada para autenticar usuários").

        O conteúdo será passado um arquivo por vez. Não é necessário lidar com múltiplos arquivos ou navegar por diretórios.

        Ignore arquivos que não sejam código-fonte (por exemplo: imagens, arquivos binários, .md, .txt, etc.) ou que não contenham lógica significativa.

        O objetivo é ajudar alguém a entender, de forma didática, o papel e funcionamento de um único arquivo de código dentro de um sistema.
        """,
        description="Agente que analisa e explica um arquivo de código por vez.",
    )


    analise = call_agent(analista_codigo, arquivo)
    return analise

In [24]:
def analisar_repositorio(repo, path="/"):
    all_analyses = []

    def _processar_arquivo(path):
        contents = repo.get_contents(path)

        # Handle if contents is a single file
        if not isinstance(contents, list):
            contents = [contents]

        for item in contents:
            if item.type == 'file':
                print(f"Analisando: {item.path}")
                try:
                    conteudo = repo.get_contents(item.path).decoded_content.decode("utf-8")
                    resposta = agente_leitor(conteudo)
                    all_analyses.append(f"## {item.path}\n\n{resposta}")
                except Exception as e:
                    all_analyses.append(f"## {item.path}\n\nErro ao analisar: {str(e)}")
            elif item.type == 'dir':
                _processar_arquivo(item.path)

    _processar_arquivo(path)
    return "\n\n---\n\n".join(all_analyses)

# Analisar todo o repositório recursivamente
print("Iniciando análise recursiva do repositório...")
resultado_completo = analisar_repositorio(repo)
print("Análise concluída. Resultado completo:\n")
display(to_markdown(resultado_completo))

Iniciando análise recursiva do repositório...
Analisando: .gitignore
Analisando: Dockerfile
Analisando: LICENSE
Analisando: README.md
Analisando: app/__init__.py
Analisando: app/extensions.py
Analisando: app/main/__init__.py
Analisando: app/main/city_codes.py
Analisando: app/main/routes.py
Analisando: app/models/weather.py
Analisando: compose.yaml
Analisando: config.py
Analisando: init.sql
Analisando: requirements.txt
Analisando: secrets/appid.env
Analisando: secrets/database_password.env
Analisando: secrets/database_url.env
Analisando: secrets/database_user.env
Analisando: test/test_app.py
Análise concluída. Resultado completo:



> ## .gitignore
> 
> Não foram encontrados arquivos de código fonte para análise. Os arquivos listados são arquivos de sistema/configuração ou diretórios, que não contêm código fonte para serem analisados.
> 
> 
> ---
> 
> ## Dockerfile
> 
> Arquivo: Dockerfile
> 
> Propósito: Este Dockerfile é usado para construir uma imagem Docker para uma aplicação Flask em Python. Ele define o ambiente, instala as dependências, copia o código-fonte e configura o comando de execução para iniciar a aplicação Flask.
> 
> Principais instruções:
> 
> - `FROM python:alpine3.18`: Define a imagem base como Python 3.18 em Alpine Linux, uma distribuição Linux leve.
> - `WORKDIR /app`: Define o diretório de trabalho dentro do contêiner como /app.
> - `ENV FLASK_APP=app`: Define a variável de ambiente FLASK_APP para "app", indicando o arquivo principal da aplicação Flask.
> - `ENV FLASK_RUN_HOST=0.0.0.0`: Define a variável de ambiente FLASK_RUN_HOST para "0.0.0.0", permitindo que a aplicação Flask seja acessível externamente.
> - `COPY requirements.txt requirements.txt`: Copia o arquivo requirements.txt para o diretório de trabalho.
> - `RUN pip install -r requirements.txt`: Instala as dependências Python listadas no arquivo requirements.txt.
> - `EXPOSE 5000`: Expõe a porta 5000, que é a porta padrão para aplicações Flask.
> - `COPY . .`: Copia todo o código-fonte do diretório atual para o diretório de trabalho no contêiner.
> - `CMD ["flask", "run"]`: Define o comando a ser executado quando o contêiner for iniciado, que inicia a aplicação Flask.
> 
> 
> ---
> 
> ## LICENSE
> 
> Este não é um arquivo de código-fonte, mas sim um texto sobre a licença de uso do código. Portanto, não há análise a ser feita.
> 
> 
> ---
> 
> ## README.md
> 
> Ok, entendi o projeto. Ele consiste em um serviço que coleta dados de clima de uma API do OpenWeather e os armazena como JSON em um banco de dados PostgreSQL. As principais funcionalidades são:
> 
> 1.  **Coleta de dados climáticos:** O serviço se conecta à API do OpenWeather, obtém informações sobre o clima de cidades específicas.
> 2.  **Armazenamento em banco de dados:** Os dados coletados são armazenados em um banco de dados PostgreSQL no formato JSON.
> 3.  **API para requisições:** O serviço expõe uma API que permite aos usuários solicitar dados climáticos para um determinado ID de usuário.
> 4.  **Progresso da requisição:** A API também permite consultar o progresso da coleta de dados climáticos para um determinado ID de usuário.
> 5.  **Testes:** O projeto inclui testes automatizados para garantir a qualidade e o funcionamento correto do serviço.
> 
> Agora, estou pronto para analisar arquivos de código individualmente e fornecer explicações mais detalhadas sobre cada um deles. Por favor, me forneça o primeiro arquivo.
> 
> 
> 
> ---
> 
> ## app/__init__.py
> 
> Arquivo: `app/__init__.py`
> 
> Propósito: Este arquivo contém a função de fábrica da aplicação Flask, responsável por criar e configurar a instância principal da aplicação.
> 
> Principais Componentes:
> 
> - `create_app(config_class=Config)`: Função que cria e configura a aplicação Flask.
>     - Inicializa a aplicação Flask.
>     - Carrega as configurações a partir de uma classe de configuração (por padrão, `Config`).
>     - Inicializa as extensões do Flask, como o banco de dados.
>     - Registra os blueprints, que são coleções de views e outros códigos relacionados que podem ser registrados em uma aplicação. No caso, registra o blueprint `main_bp` do módulo `app.main`.
> 
> Relação com o Sistema:
> Este arquivo é o ponto de entrada para a criação da aplicação Flask. Ele configura a aplicação, inicializa extensões e registra blueprints, preparando a aplicação para receber e responder a requisições HTTP. A função `create_app` é normalmente chamada no script principal para iniciar a aplicação.
> 
> 
> ---
> 
> ## app/extensions.py
> 
> O arquivo define uma instância do SQLAlchemy para uso com o Flask.
> 
> -   **Nome do arquivo:** Não especificado (o código é um fragmento)
> -   **Propósito:** Configurar a extensão SQLAlchemy para interagir com um banco de dados em uma aplicação Flask.
> -   **Principais elementos:**
>     -   `db = SQLAlchemy()`: Cria uma instância do SQLAlchemy, que será usada para definir modelos de banco de dados e realizar operações no banco de dados. Essa instância precisa ser associada a uma aplicação Flask para funcionar corretamente.
> 
> Em um sistema típico, essa instância `db` seria usada para definir as tabelas do banco de dados como classes Python (modelos), e as operações de banco de dados (criar, ler, atualizar, deletar) seriam realizadas através dessa instância.
> 
> 
> ---
> 
> ## app/main/__init__.py
> 
> O arquivo `__init__.py` dentro do diretório `app/main` define um Blueprint chamado `main` para o Flask. Blueprints são usados para organizar um grupo de rotas e outros componentes relacionados em uma aplicação Flask.
> 
> - `bp = Blueprint('main', __name__)`: Cria um novo Blueprint chamado 'main'. O `__name__` garante que o Blueprint saiba onde seus recursos estão localizados.
> 
> - `from app.main import routes`: Importa o módulo `routes.py` associado a este Blueprint. A importação de `routes` geralmente contém as definições de rota (URLs) que este Blueprint irá gerenciar.
> 
> Em resumo, este arquivo inicializa um Blueprint para o módulo principal da aplicação, preparando-o para registrar rotas e views que lidam com a lógica principal da aplicação web. Ele estabelece a estrutura para organizar as funcionalidades principais da aplicação em um módulo coeso e reutilizável.
> 
> 
> 
> ---
> 
> ## app/main/city_codes.py
> 
> O arquivo contém uma lista de inteiros chamada `city_codes`. Essa lista provavelmente armazena códigos de identificação de cidades, possivelmente para serem usados em alguma consulta a uma base de dados ou API de previsão do tempo. Cada número inteiro representa um código único associado a uma cidade específica.
> 
> 
> ---
> 
> ## app/main/routes.py
> 
> ```text
> Arquivo: app/main/routes.py
> 
> Propósito:
> Este arquivo define as rotas principais da aplicação Flask, responsáveis por lidar com requisições relacionadas a informações meteorológicas. Ele inclui rotas para buscar dados meteorológicos associados a um usuário e para criar ou atualizar esses dados.
> 
> Principais Funções e Componentes:
> 
> - Definição da Blueprint: Cria um blueprint chamado bp (do módulo app.main), que organiza as rotas relacionadas à funcionalidade principal da aplicação.
> 
> - Rota principal ("/"):
>     - GET: Recupera informações meteorológicas para um determinado user_id. Busca no banco de dados um registro Weather associado ao user_id fornecido e retorna os dados meteorológicos (json_data) e a porcentagem de cidades com dados coletados.
>     - POST: Cria ou atualiza informações meteorológicas para um user_id. Recebe o user_id via formulário, itera sobre uma lista de city_codes, faz requisições à API OpenWeatherMap para cada cidade e salva/atualiza os dados no banco de dados. Implementa um sistema de rate limiting para evitar sobrecarga na API.
> 
> - Função rate_limit(limit, interval): Implementa um mecanismo de rate limiting para controlar a frequência das requisições à API OpenWeatherMap. Suspende a execução se o número de requisições exceder o limite definido dentro de um determinado intervalo de tempo.
> 
> - Função run_city(user_id, timestamp, city_id): Função assíncrona que faz a requisição à API OpenWeatherMap para uma cidade específica, processa a resposta e atualiza o banco de dados. Utiliza a biblioteca httpx para realizar requisições assíncronas.
> 
> - Interação com o Banco de Dados: Utiliza o modelo Weather (do módulo app.models.weather) para interagir com o banco de dados. Realiza operações de consulta (Weather.query.filter_by), criação (Weather(...)), atualização (db.session.query(...).update(...)) e commit (db.session.commit()).
> 
> - Integração com a API OpenWeatherMap: Faz requisições para a API OpenWeatherMap para obter dados meteorológicos em tempo real. Utiliza a chave de API armazenada na variável de ambiente APPID.
> 
> Relação com Outras Partes do Sistema:
> 
> - Modelos (app/models/weather.py): O arquivo interage diretamente com o modelo Weather, que define a estrutura dos dados meteorológicos no banco de dados.
> - Banco de Dados: As informações meteorológicas são armazenadas e recuperadas do banco de dados, utilizando as funções do Flask-SQLAlchemy.
> - Códigos de Cidades (app/main/city_codes.py): Utiliza uma lista de códigos de cidades (city_codes) para iterar e buscar informações meteorológicas de várias localidades.
> - Configurações (variáveis de ambiente): A chave da API OpenWeatherMap (APPID) é obtida a partir de uma variável de ambiente.
> - Blueprint (app/main/__init__.py): As rotas definidas neste arquivo são registradas em um blueprint, que é então registrado na aplicação Flask principal.
> 
> 
> 
> 
> ---
> 
> ## app/models/weather.py
> 
> O arquivo `Weather` define um modelo de banco de dados para armazenar informações sobre o clima.
> 
> - **Nome do Arquivo:** `Weather`
> - **Propósito:** Este arquivo define a estrutura da tabela `Weather` no banco de dados, que será usada para armazenar dados relacionados ao clima.
> - **Principais Componentes:**
>     - `Weather` (classe): Define a estrutura da tabela `Weather`.
>         - `user_id` (coluna): Chave primária do tipo inteiro, provavelmente usada para identificar o usuário associado aos dados climáticos.
>         - `date_time` (coluna): Armazena a data e hora da medição climática.
>         - `json_data` (coluna): Armazena os dados climáticos em formato JSON, permitindo flexibilidade para armazenar diferentes tipos de informações (temperatura, umidade, etc.).
> - **Relação com o Sistema:** Este modelo provavelmente será usado em conjunto com outros componentes do sistema para coletar, armazenar e exibir informações sobre o clima. Por exemplo, uma função poderia consultar este modelo para obter dados climáticos para um determinado usuário e data/hora.
> 
> 
> ---
> 
> ## compose.yaml
> 
> O arquivo `docker-compose.yml` define a configuração para orquestrar dois serviços Docker: `app` e `db`.
> 
> - **app:** Este serviço constrói a aplicação a partir do diretório local (definido por `./`), mapeia a porta 8000 do host para a porta 5000 do contêiner, carrega variáveis de ambiente de arquivos `*.env` e declara uma dependência no serviço `db`.  A aplicação, portanto, depende do banco de dados para funcionar.
> 
> - **db:** Este serviço usa a imagem `postgres:15.4` para criar um contêiner PostgreSQL. Define o banco de dados padrão como `flask_app_db`. Executa um script SQL para inicializar o banco de dados, usando o arquivo `init.sql`. Carrega variáveis de ambiente de arquivos `*.env` para configurar o usuário e senha do banco de dados.
> 
> Em resumo, este arquivo `docker-compose.yml` configura um ambiente de desenvolvimento ou produção para uma aplicação que depende de um banco de dados PostgreSQL. Ele define como os contêineres são construídos, quais portas são expostas, como as variáveis de ambiente são configuradas e quais dependências existem entre os serviços.
> 
> 
> ---
> 
> ## config.py
> 
> O arquivo `config.py` define uma classe de configuração chamada `Config`. Essa classe é usada para armazenar as configurações da aplicação, como a chave secreta e o URI do banco de dados. As configurações são obtidas das variáveis de ambiente, o que permite que a aplicação seja configurada de forma diferente em diferentes ambientes (por exemplo, desenvolvimento, produção, teste).
> 
> **Principais elementos:**
> 
> *   **`SECRET_KEY`**: Chave secreta usada para assinar cookies de sessão, prevenindo modificações maliciosas. A chave é obtida da variável de ambiente `SECRET_KEY`.
> *   **`SQLALCHEMY_DATABASE_URI`**: URI de conexão do banco de dados SQLAlchemy. A URI é obtida da variável de ambiente `DATABASE_URL`.
> *   **`SQLALCHEMY_TRACK_MODIFICATIONS`**: Configuração para habilitar ou desabilitar o rastreamento de modificações de objetos pelo SQLAlchemy. Está definida como `False` para evitar sobrecarga desnecessária.
> 
> Essa classe de configuração é normalmente usada para inicializar a aplicação Flask, fornecendo os parâmetros necessários para o funcionamento correto da aplicação, incluindo a conexão com o banco de dados e a segurança da sessão. Por exemplo, a `SECRET_KEY` pode ser usada para autenticar usuários.
> 
> 
> ---
> 
> ## init.sql
> 
> O arquivo contém a definição de uma tabela de banco de dados chamada `weather`. Essa tabela armazena dados relacionados ao clima, associados a um usuário específico em um determinado momento.
> 
> - A coluna `user_id` é um inteiro que identifica o usuário associado aos dados climáticos.
> - A coluna `date_time` armazena a data e hora em que os dados climáticos foram registrados.
> - A coluna `json_data` armazena os dados climáticos em formato JSON, permitindo flexibilidade para armazenar diferentes tipos de informações climáticas.
> - A chave primária da tabela é `user_id`, o que garante que cada usuário tenha apenas um registro de dados climáticos.
> 
> Essa tabela pode ser usada em um sistema para armazenar e consultar informações climáticas para diferentes usuários. Por exemplo, um aplicativo de previsão do tempo poderia usar essa tabela para armazenar os dados climáticos coletados de diferentes fontes e associá-los aos usuários que consultam as informações.
> 
> 
> 
> ---
> 
> ## requirements.txt
> 
> Este não é um arquivo de código-fonte. É uma lista de dependências (pacotes) usadas em um projeto Python, especificando as versões exatas de cada pacote. O arquivo provavelmente se chama `requirements.txt` ou similar. Ele serve para garantir que o ambiente de desenvolvimento e produção tenham as mesmas versões das bibliotecas, evitando incompatibilidades.
> 
> As dependências listadas indicam que o projeto provavelmente é uma aplicação web construída com o framework Flask, utilizando um banco de dados relacional (provavelmente PostgreSQL) através do SQLAlchemy. As bibliotecas `requests` e `httpx` são usadas para fazer requisições HTTP a outros serviços/APIs.
> 
> 
> 
> ---
> 
> ## secrets/appid.env
> 
> Este arquivo provavelmente contém a chave de identificação (APPID) para acessar uma API externa, como um serviço de previsão do tempo ou outro serviço baseado em API. Ele serve para configurar a aplicação com a credencial necessária para autenticação e autorização ao utilizar a API.
> 
> 
> ---
> 
> ## secrets/database_password.env
> 
> Este arquivo provavelmente é um arquivo de configuração, possivelmente um arquivo `.env`. Ele define uma variável de ambiente chamada `POSTGRES_PASSWORD`. Essa variável é usada para armazenar a senha do banco de dados PostgreSQL. Em um sistema, esse arquivo permite configurar a senha do banco de dados de forma flexível, sem codificá-la diretamente no código-fonte. Isso é importante para segurança e para facilitar a implantação em diferentes ambientes (desenvolvimento, teste, produção), onde as senhas podem variar.
> 
> 
> ---
> 
> ## secrets/database_url.env
> 
> Este arquivo `.env` define a variável de ambiente `DATABASE_URL`. Esta variável é comumente usada em aplicações para especificar a string de conexão com o banco de dados. A string de conexão contém informações como o tipo de banco de dados, endereço do servidor, nome do banco de dados, usuário e senha, permitindo que a aplicação se conecte e interaja com o banco de dados.
> 
> 
> 
> ---
> 
> ## secrets/database_user.env
> 
> Este arquivo de configuração contém a definição da variável de ambiente `POSTGRES_USER`, que provavelmente é usada para especificar o nome de usuário padrão para se conectar a um banco de dados PostgreSQL. Em um sistema, essa variável seria usada para configurar a autenticação ao acessar o banco de dados, seja em scripts de inicialização, configurações de aplicativos ou conexões diretas ao banco de dados.
> 
> 
> ---
> 
> ## test/test_app.py
> 
> ```text
> Arquivo: test.py
> 
> Este arquivo contém testes automatizados para uma API que opera em 'localhost:8000'.
> 
> Principais Componentes:
> 
> 1.  Classe `TestAPI`:
>     *   É uma classe de teste que herda de `unittest.TestCase`. Isso significa que ela contém métodos que testam diferentes aspectos da API.
> 
> 2.  Método `test_post`:
>     *   Testa o endpoint da API que responde a requisições POST.
>     *   Envia dados (`user_id=4`) via POST para a URL da API.
>     *   Verifica se o código de status da resposta é 200 (OK).
>     *   Compara o resultado da resposta com um resultado esperado (`correct_result`), validando se a API retorna os dados corretos para o `user_id` fornecido.
>     *   Verifica se o campo `timestamp` existe na resposta, indicando que a API está retornando informações de data/hora.
> 
> 3.  Método `test_get`:
>     *   Testa o endpoint da API que responde a requisições GET.
>     *   Envia uma requisição GET para a URL da API com o parâmetro `user_id=1`.
>     *   Verifica se o código de status da resposta é 200 (OK).
>     *   Compara o resultado da resposta com um resultado esperado (`correct_result`), confirmando se a API retorna a porcentagem correta para o `user_id` fornecido.
> 
> Em resumo, este script usa a biblioteca `unittest` para definir e executar testes que verificam se a API em `http://localhost:8000` está funcionando corretamente para requisições POST e GET, validando tanto o código de status HTTP quanto os dados retornados. Ele simula requisições de clientes e compara as respostas da API com valores esperados.
> ```


In [40]:
##########################################
# --- Agente 2: Agente analista de projetos --- #
##########################################
def agente_analista(descricao_codigo):
    avaliador_projeto = Agent(
        name="avaliador_projeto",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em análise de projetos de software. Sua tarefa é analisar a estrutura e os componentes
        de um projeto de código com base em uma listagem em Markdown contendo:

        - O nome de cada arquivo no projeto.
        - Uma breve descrição do conteúdo ou função de cada arquivo.

        Com base nessas informações, forneça uma **avaliação geral do projeto**, seguindo o seguinte formato:

        1. **Descrição geral do projeto**:
        Explique o que o projeto parece fazer, qual é o seu propósito, e para que tipo de aplicação ele é voltado.

        2. **Pontos fortes**:
        Liste aspectos positivos como organização da estrutura, separação de responsabilidades, clareza nos nomes dos arquivos,
        presença de testes, documentação, boas práticas visíveis, modularização etc.

        3. **Pontos fracos**:
        Aponte aspectos que possam ser melhorados, como ausência de testes, acoplamento excessivo, falta de estrutura de pastas,
        arquivos muito genéricos ou confusos, ausência de padrões claros etc.

        4. **Recomendações e próximos passos**:
        Sugira o que deveria ser feito para evoluir o projeto — como reorganização, documentação adicional, melhorias técnicas,
        refatoração, adição de testes, melhorias de segurança, integração contínua, etc.

        Use um tom analítico e profissional, como um revisor técnico que deseja ajudar a melhorar o projeto.

        O conteúdo será fornecido em formato Markdown, com a listagem dos arquivos e suas descrições.
        Não assuma nada além do que está presente nesse conteúdo.
        """,
        description="Agente que avalia a qualidade geral de um projeto de software com base em uma listagem de arquivos e descrições.",
    )

    analise = call_agent(avaliador_projeto, descricao_codigo)
    return analise

In [41]:
resultado = agente_analista(resultado_completo)
print("\n--- 📝 Resultado do Agente 2 (Analista) ---\n")
display(to_markdown(resultado))
print("--------------------------------------------------------------")


--- 📝 Resultado do Agente 2 (Analista) ---



> ## Avaliação do Projeto
> 
> Com base na análise dos arquivos fornecidos, segue uma avaliação geral do projeto:
> 
> ### 1. Descrição geral do projeto:
> 
> O projeto consiste em um serviço que coleta dados climáticos de uma API externa (OpenWeather) e os armazena em um banco de dados PostgreSQL no formato JSON. Ele expõe uma API para que os usuários possam solicitar dados climáticos associados a um ID de usuário específico e verificar o progresso da coleta de dados. O projeto utiliza o framework Flask, SQLAlchemy para interação com o banco de dados, e bibliotecas como `requests` e `httpx` para fazer requisições HTTP. A configuração é feita através de variáveis de ambiente, o que facilita a implantação em diferentes ambientes. O projeto também inclui testes automatizados.
> 
> ### 2. Pontos fortes:
> 
> *   **Organização:** O projeto segue uma estrutura de diretórios razoável (com `app`, `app/main`, `app/models`, `test`).
> *   **Separação de Responsabilidades:** A aplicação utiliza blueprints para organizar as rotas (`app/main/routes.py`). A configuração é separada do código (arquivos `.env`, `config.py`).
> *   **Uso de Padrões:** O projeto utiliza uma função de fábrica para criar a aplicação Flask (`app/__init__.py`).
> *   **Testes:** A presença de testes automatizados (`test/test_app.py`) é um ponto forte, indicando uma preocupação com a qualidade do código.
> *   **Gerenciamento de Dependências:** O uso de `requirements.txt` garante a reprodutibilidade do ambiente.
> *   **Dockerização:** O projeto é dockerizado, facilitando a implantação e o isolamento do ambiente.
> *   **Configuração:** O uso de variáveis de ambiente para configurar a aplicação (chaves de API, credenciais do banco de dados) é uma boa prática de segurança e facilita a implantação em diferentes ambientes.
> *   **Rate Limiting:** Implementação de rate limiting para evitar sobrecarga na API do OpenWeatherMap.
> *   **Armazenamento de dados:** Armazenamento dos dados da API em formato JSON permite maior flexibilidade e preservação da estrutura dos dados originais.
> 
> ### 3. Pontos fracos:
> 
> *   **Segurança:** As senhas do banco de dados e chaves da API estão sendo versionadas no repositório, mesmo que em arquivos separados. Isso é um risco de segurança.
> *   **Tratamento de erros:** Não há menção de tratamento de erros nas requisições à API externa. Falhas na API podem interromper o processo de coleta de dados.
> *   **Documentação:** A documentação do projeto poderia ser mais completa, explicando como configurar e usar a aplicação.
> *   **Tipagem:** O código não utiliza tipagem estática (type hints). A adição de type hints melhoraria a legibilidade e a manutenção do código.
> *   **Nomes genéricos:** O nome do arquivo de testes (`test.py`) é muito genérico. Um nome mais descritivo, como `test_api.py` ou `test_weather_api.py`, seria melhor.
> *   **Falta de abstração:** A lógica de acesso ao banco de dados está diretamente nas rotas. Poderia ser criada uma camada de serviço para abstrair essa lógica.
> *   **Código assíncrono:** A implementação de requisições assíncronas poderia ser melhorada com um gerenciador de tarefas (como Celery) para lidar com a coleta de dados em segundo plano.
> 
> ### 4. Recomendações e próximos passos:
> 
> *   **Segurança:**
>     *   **NÃO** versionar arquivos com senhas ou chaves de API. Usar segredos gerenciados por um serviço de nuvem (como AWS Secrets Manager, Google Cloud Secret Manager, Azure Key Vault) ou um sistema de gerenciamento de segredos como HashiCorp Vault.
>     *   Implementar um sistema de autenticação e autorização para a API, garantindo que apenas usuários autorizados possam acessar os dados.
> *   **Tratamento de erros:**
>     *   Adicionar tratamento de erros nas requisições à API externa (try-except blocks) para lidar com falhas na API ou problemas de rede.
>     *   Implementar logging para registrar erros e outros eventos importantes.
> *   **Documentação:**
>     *   Adicionar um guia de instalação e configuração no `README.md`.
>     *   Documentar a API, explicando os endpoints, parâmetros e formatos de resposta.
> *   **Tipagem:**
>     *   Adicionar type hints ao código para melhorar a legibilidade e a manutenção.
> *   **Refatoração:**
>     *   Mover a lógica de acesso ao banco de dados para uma camada de serviço.
>     *   Usar um gerenciador de tarefas (como Celery) para lidar com a coleta de dados em segundo plano.
> *   **Testes:**
>     *   Adicionar mais testes para cobrir diferentes cenários (erros, casos de borda, etc.).
>     *   Implementar testes de integração para verificar a interação entre os diferentes componentes do sistema.
> *   **Monitoramento:**
>     *   Implementar um sistema de monitoramento para acompanhar a saúde da aplicação e identificar problemas em tempo real.
> 
> Em resumo, o projeto tem uma base sólida e bem organizada, mas precisa de algumas melhorias em segurança, tratamento de erros, documentação e testes para se tornar mais robusto e fácil de manter.
> 


--------------------------------------------------------------


In [45]:
# criar uma branch chamada evaluation, adicione o resultado em um arquivo evaluation.md e faça um PR para branch main

# Define o nome da branch e do arquivo de avaliação
branch_name = "evaluation"
evaluation_filename = "evaluation.md"

# Obtém o usuário autenticado
user = g.get_user()

# Obtém o repositório novamente (para garantir que temos o objeto mais recente)
# Assumindo que 'repo' já está definido no código anterior como o objeto do repositório
# repo = user.get_repo("OpenWeatherService") # Descomente e ajuste o nome do repo se necessário

try:
    # Verifica se a branch de avaliação já existe
    try:
        ref = repo.get_git_ref(f"heads/{branch_name}")
        print(f"Branch '{branch_name}' já existe. Atualizando...")
    except Exception as e:
        # Se a branch não existe, cria uma nova a partir da branch principal (main)
        print(f"Branch '{branch_name}' não encontrada. Criando nova branch...")
        main_branch = repo.get_branch("main")
        repo.create_git_ref(ref=f"refs/heads/{branch_name}", sha=main_branch.commit.sha)
        print(f"Branch '{branch_name}' criada com sucesso.")
        ref = repo.get_git_ref(f"heads/{branch_name}") # Obtém a referência da nova branch

    # Verifica se o arquivo evaluation.md já existe na branch de avaliação
    try:
        contents = repo.get_contents(evaluation_filename, ref=branch_name)
        # Atualiza o arquivo existente
        print(f"Arquivo '{evaluation_filename}' encontrado na branch '{branch_name}'. Atualizando...")
        repo.update_file(
            path=evaluation_filename,
            message=f"Atualiza a avaliação do projeto",
            content=resultado,
            sha=contents.sha,
            branch=branch_name
        )
        print(f"Arquivo '{evaluation_filename}' atualizado com sucesso.")
    except Exception as e:
         # Se o arquivo não existe, cria um novo
        print(f"Arquivo '{evaluation_filename}' não encontrado na branch '{branch_name}'. Criando novo arquivo...")
        repo.create_file(
            path=evaluation_filename,
            message=f"Adiciona a avaliação inicial do projeto",
            content=resultado,
            branch=branch_name
        )
        print(f"Arquivo '{evaluation_filename}' criado com sucesso.")

    # Cria o Pull Request (PR)
    print(f"Criando Pull Request da branch '{branch_name}' para 'main'...")
    try:
        pr = repo.create_pull(
            title=f"Avaliação automática do projeto",
            body=f"Este Pull Request adiciona/atualiza o arquivo `{evaluation_filename}` com a avaliação automática do projeto gerada em {data_de_hoje}.",
            head=branch_name,
            base="main"
        )
        print(f"Pull Request criado com sucesso: {pr.html_url}")
    except Exception as e:
        print(f"Erro ao criar o Pull Request: {e}")
        # Verifica se já existe um PR da mesma branch para base
        open_prs = repo.get_pulls(state='open', head=f'{user.login}:{branch_name}', base='main')
        if open_prs.totalCount > 0:
            print("Já existe um Pull Request aberto desta branch para 'main'.")
            print(f"PR existente: {open_prs[0].html_url}")
        else:
             print("Ocorreu um erro inesperado ao criar o PR.")


except Exception as e:
    print(f"Ocorreu um erro geral: {e}")

# Fechar a conexão do Github ao final
g.close()


Branch 'evaluation' já existe. Atualizando...
Arquivo 'evaluation.md' encontrado na branch 'evaluation'. Atualizando...
Arquivo 'evaluation.md' atualizado com sucesso.
Criando Pull Request da branch 'evaluation' para 'main'...
Erro ao criar o Pull Request: name 'data_de_hoje' is not defined
Já existe um Pull Request aberto desta branch para 'main'.
PR existente: https://github.com/heitorcfelix/OpenWeatherService/pull/1


In [74]:
##########################################
# --- Agente 3: Assistente de códigos --- #
##########################################
def agente_programador(codigo):
    assistente_codigo = Agent(
        name="assistente_codigo",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de melhoria de código. Receberá como entrada um único arquivo de código (como .py ou .ipynb convertido para texto)
        e deve retornar uma **versão aprimorada do código original**.

        Seu objetivo é aplicar melhorias com base em:

        - Boas práticas de Clean Code
        - Comentários úteis e objetivos onde fizer sentido
        - Docstrings explicativas em classes e funções, se necessário
        - Nomes de variáveis e funções mais descritivos (evitar nomes genéricos)
        - Estilo de formatação consistente (ex: PEP8 para Python)

        **IMPORTANTE:**
        - A saída deve conter **somente o código Python melhorado**.
        - Não inclua qualquer explicação, justificativa ou comentários fora do código.
        - Não use blocos Markdown (ex: ```python).
        - Não escreva frases como “Melhorias sugeridas” ou “Código atualizado”.
        - Apenas retorne o código limpo, direto, pronto para ser usado.

        ### Exemplo

        #### Entrada:
        class Weather(db.Model):
            user_id = db.Column(db.Integer, primary_key=True)
            date_time = db.Column(db.DateTime)
            json_data = db.Column(db.JSON)

        #### Saída esperada:
        class Weather(db.Model):
            '''
            Modelo para armazenar dados meteorológicos.

            Atributos:
                user_id (int): Identificador do usuário.
                date_time (datetime): Data e hora da coleta.
                json_data (JSON): Dados meteorológicos coletados.
            '''
            user_id = db.Column(db.Integer, primary_key=True)
            date_time = db.Column(db.DateTime)
            json_data = db.Column(db.JSON)
        """,
        description="Agente que reescreve código Python (ou notebooks) aplicando boas práticas — saída somente com o código puro.",
    )

    analise = call_agent(assistente_codigo, codigo)
    return analise

In [78]:
from ipywidgets import Dropdown, Output, Button
from IPython.display import Markdown, display

# Variável global para armazenar as sugestões do agente
ultima_sugestao_agente = None

# Obter a lista de arquivos do repositório
arquivos_repo = _listar_todos_os_arquivos(repo)
# Criar uma lista de nomes de arquivos para o dropdown
nomes_arquivos = [arq.path for arq in arquivos_repo]

# Criar o Dropdown
dropdown_arquivos = Dropdown(
    options=nomes_arquivos,
    description='Escolha um arquivo:',
    disabled=False,
)

# Área de saída para exibir o resultado
output_area = Output()

# Função para lidar com a seleção do Dropdown
def on_value_change(change):
    global ultima_sugestao_agente

    with output_area:
        output_area.clear_output()
        selected_file_path = change['new']
        print(f"Arquivo selecionado: {selected_file_path}")

        try:
            # Obter o conteúdo do arquivo selecionado
            file_content = repo.get_contents(selected_file_path).decoded_content.decode("utf-8")
            print("Conteúdo do arquivo lido.")

            # Submeter o conteúdo para o agente_programador
            print("Submetendo conteúdo para o agente_programador...")
            sugestoes_agente = agente_programador(file_content)
            ultima_sugestao_agente = sugestoes_agente  # Armazenar na variável global
            print("Sugestões finalizadas pelo agente programador")
            print(sugestoes_agente)

        except Exception as e:
            display(Markdown(f"Ocorreu um erro ao processar o arquivo: {e}"))
            ultima_sugestao_agente = None

# Registrar a função para ser chamada quando o valor do Dropdown mudar
dropdown_arquivos.observe(on_value_change, names='value')

# Exibir o Dropdown e a área de saída
display(dropdown_arquivos, output_area)

# Função para obter a última sugestão do agente
def obter_ultima_sugestao():
    return ultima_sugestao_agente


Dropdown(description='Escolha um arquivo:', options=('.gitignore', 'Dockerfile', 'LICENSE', 'README.md', 'comp…

Output()

In [80]:
# Função para criar um Pull Request com a última sugestão do agente
def criar_pull_request():
    global ultima_sugestao_agente

    if ultima_sugestao_agente is None:
        display(Markdown("Não há sugestões disponíveis. Por favor, selecione um arquivo primeiro."))
        return

    try:
        # Obter o arquivo selecionado
        selected_file_path = dropdown_arquivos.value

        # Obter o conteúdo atual do arquivo
        file_content = repo.get_contents(selected_file_path).decoded_content.decode("utf-8")

        # Aplicar a sugestão do agente ao conteúdo do arquivo
        novo_conteudo = ultima_sugestao_agente

        # Criar um novo branch
        branch_name = f"sugestao-agente"
        source_branch = repo.get_branch("main")  # ou master, dependendo do repositório
        repo.create_git_ref(ref=f"refs/heads/{branch_name}", sha=source_branch.commit.sha)

        # Fazer commit da alteração no novo branch
        commit_message = "Aplicar sugestão do agente programador"
        repo.update_file(
            path=selected_file_path,
            message=commit_message,
            content=novo_conteudo,
            sha=repo.get_contents(selected_file_path, ref=branch_name).sha,
            branch=branch_name
        )

        # Criar o Pull Request
        pr = repo.create_pull(
            title=f"Sugestão do agente para {selected_file_path}",
            body="Este PR contém sugestões geradas automaticamente pelo agente programador.",
            head=branch_name,
            base="main"  # ou master, dependendo do repositório
        )

        display(Markdown(f"✅ Pull Request criado com sucesso: [PR #{pr.number}]({pr.html_url})"))

    except Exception as e:
        display(Markdown(f"❌ Erro ao criar Pull Request: {str(e)}"))

# Botão para criar o Pull Request
botao_criar_pr = Button(
    description='Criar Pull Request',
    button_style='success',
    tooltip='Criar um PR com a última sugestão do agente'
)

# Conectar o botão à função
botao_criar_pr.on_click(lambda b: criar_pull_request())

# Exibir o botão
display(botao_criar_pr)


Button(button_style='success', description='Criar Pull Request', style=ButtonStyle(), tooltip='Criar um PR com…

✅ Pull Request criado com sucesso: [PR #2](https://github.com/heitorcfelix/OpenWeatherService/pull/2)